In [1]:
import pandas as pd
import numpy as np

from classes.KAnonymizer import KAnonymizer

In [2]:
ds = pd.read_csv(filepath_or_buffer='Artists-Pseudo-02.csv',sep=',')
ds

,Unnamed: 0,ConstituentID,DisplayName,Region,Gender,BeginDate,Wiki QID,Income ($)
0,0,1,Robert Arneson,Port Vila; Vanuatu; Melanesia; Oceania,Male,1930,Q6113656,20900
1,1,2,Doroteo Arnaiz,Fort-de-France; Martinique; Caribbean; Americas,Male,1936,Q164351,9300
2,2,3,Bill Arnold,Sanaa; Yemen; Western Asia; Asia,Male,1941,Q6668231,5400
3,3,4,Charles Arnoldi,Lome; Togo; Western Africa; Africa,Male,1946,Q1063584,11600
4,4,5,Per Arnoldi,Luanda; Angola; Middle Africa; Africa,Male,1941,Q663175,9300
...,...,...,...,...,...,...,...,...
15710,15710,132165,Eric Klarenbeek,Kuala Lumpur; Malaysia; South-Eastern Asia; Asia,Male,1978,Q1471465,9300
15711,15711,132166,Maartje Dros,The Valley; Anguilla; Caribbean; Americas,Female,1862,Q259594,9300
15712,15712,132170,The Event Horizon Telescope Collaboration,Maseru; Lesotho; Southern Africa; Africa,male,1791,Q182014,34400
15713,15713,132427,Krasnyi proletarii,Accra; Ghana; Western Africa; Africa,Female,1821,Q380494,12400


In [3]:
sensitivity_features = [i  for i in ds.columns if i not in ['Region','BeginDate','Income ($)']] 
sensitivity_features

['Unnamed: 0', 'ConstituentID', 'DisplayName', 'Gender', 'Wiki QID']

In [449]:
df = KAnonymizer(ds,sensitivity_features=sensitivity_features)

In [450]:
df = df.getAnonymizedData()
df

,Income ($),city,country,sub-region,region,year,decade,century
0,20900,Port Vila,Vanuatu,Melanesia,Oceania,1930,1920,20
1,9300,Fort-de-France,Martinique,Caribbean,Americas,1936,1930,20
2,5400,Sanaa,Yemen,Western Asia,Asia,1941,1940,20
3,11600,Lome,Togo,Western Africa,Africa,1946,1940,20
4,9300,Luanda,Angola,Middle Africa,Africa,1941,1940,20
...,...,...,...,...,...,...,...,...
15710,9300,Kuala Lumpur,Malaysia,South-Eastern Asia,Asia,1978,1970,20
15711,9300,The Valley,Anguilla,Caribbean,Americas,1862,1860,19
15712,34400,Maseru,Lesotho,Southern Africa,Africa,1791,1790,18
15713,12400,Accra,Ghana,Western Africa,Africa,1821,1820,19


In [451]:
K_ANONIM = 8

In [452]:
contagem_df = df.groupby(['city', 'century'])['Income ($)'].count()
contagem_df

city       century
Abu Dhabi  18          2
           19         12
           20         44
           21          3
Abuja      18          1
                      ..
Yerevan    21          2
Zagreb     18          1
           19         19
           20         48
           21          2
Name: Income ($), Length: 827, dtype: int64

In [453]:
val_a_sub = contagem_df[contagem_df < K_ANONIM].index
val_a_sub

MultiIndex([(   'Abu Dhabi', 18),
            (   'Abu Dhabi', 21),
            (       'Abuja', 18),
            (       'Abuja', 21),
            (       'Accra', 18),
            (       'Accra', 21),
            (   'Adamstown', 21),
            ( 'Addis Ababa', 18),
            ( 'Addis Ababa', 21),
            (       'Alofi', 21),
            ...
            (  'Willemstad', 21),
            (    'Windhoek', 21),
            ('Yamoussoukro', 18),
            ('Yamoussoukro', 21),
            (       'Yaren', 18),
            (       'Yaren', 21),
            (     'Yerevan', 18),
            (     'Yerevan', 21),
            (      'Zagreb', 18),
            (      'Zagreb', 21)],
           names=['city', 'century'], length=346)

for (city, decade) in val_a_sub:
    novo_valor_year = df.loc[(df['city'] == city) & (df['decade'] == decade), 'century'].values[0]
    print(decade,novo_valor_year)
    df.loc[(df['city'] == city) & (df['decade'] == decade), 'decade'] = novo_valor_year


In [454]:
df

,Income ($),city,country,sub-region,region,year,decade,century
0,20900,Port Vila,Vanuatu,Melanesia,Oceania,1930,1920,20
1,9300,Fort-de-France,Martinique,Caribbean,Americas,1936,1930,20
2,5400,Sanaa,Yemen,Western Asia,Asia,1941,1940,20
3,11600,Lome,Togo,Western Africa,Africa,1946,1940,20
4,9300,Luanda,Angola,Middle Africa,Africa,1941,1940,20
...,...,...,...,...,...,...,...,...
15710,9300,Kuala Lumpur,Malaysia,South-Eastern Asia,Asia,1978,1970,20
15711,9300,The Valley,Anguilla,Caribbean,Americas,1862,1860,19
15712,34400,Maseru,Lesotho,Southern Africa,Africa,1791,1790,18
15713,12400,Accra,Ghana,Western Africa,Africa,1821,1820,19


In [455]:
nova_contagem_df = df.groupby(['city', 'century'])['Income ($)'].count()
nova_contagem_df

city       century
Abu Dhabi  18          2
           19         12
           20         44
           21          3
Abuja      18          1
                      ..
Yerevan    21          2
Zagreb     18          1
           19         19
           20         48
           21          2
Name: Income ($), Length: 827, dtype: int64

In [456]:
novos_val_a_sub = nova_contagem_df[nova_contagem_df < K_ANONIM].index
novos_val_a_sub

MultiIndex([(   'Abu Dhabi', 18),
            (   'Abu Dhabi', 21),
            (       'Abuja', 18),
            (       'Abuja', 21),
            (       'Accra', 18),
            (       'Accra', 21),
            (   'Adamstown', 21),
            ( 'Addis Ababa', 18),
            ( 'Addis Ababa', 21),
            (       'Alofi', 21),
            ...
            (  'Willemstad', 21),
            (    'Windhoek', 21),
            ('Yamoussoukro', 18),
            ('Yamoussoukro', 21),
            (       'Yaren', 18),
            (       'Yaren', 21),
            (     'Yerevan', 18),
            (     'Yerevan', 21),
            (      'Zagreb', 18),
            (      'Zagreb', 21)],
           names=['city', 'century'], length=346)

In [457]:
for (city, century) in novos_val_a_sub:
    novo_valor_city = df.loc[(df['city'] == city) & (df['century'] == century), 'country'].values[0]
    print(city,novo_valor_city)
    df.loc[(df['city'] == city) & (df['century'] == century), 'city'] = novo_valor_city

Abu Dhabi  United Arab Emirates
Abu Dhabi  United Arab Emirates
Abuja  Nigeria
Abuja  Nigeria
Accra  Ghana
Accra  Ghana
Adamstown  Pitcairn Island
Addis Ababa  Ethiopia
Addis Ababa  Ethiopia
Alofi  Niue
Amman  Jordan
Amman  Jordan
Amsterdam  Netherlands The
Andorra la Vella  Andorra
Andorra la Vella  Andorra
Ankara  Turkey
Antananarivo  Madagascar
Antananarivo  Madagascar
Apia  Samoa
Ashgabat  Turkmenistan
Asuncion  Paraguay
Athens  Greece
Avarua  Cook Islands
Baghdad  Iraq
Baghdad  Iraq
Baku  Azerbaijan
Baku  Azerbaijan
Bamako  Mali
Bamako  Mali
Bandar Seri Begawan  Brunei
Bandar Seri Begawan  Brunei
Bangkok  Thailand
Bangui  Central African Republic
Basse-Terre  Guadeloupe
Basseterre  Saint Kitts And Nevis
Basseterre  Saint Kitts And Nevis
Beijing  China
Beijing  China
Beirut  Lebanon
Belgrade  Serbia
Belgrade  Serbia
Belmopan  Belize
Belmopan  Belize
Berlin  Germany
Berlin  Germany
Bern  Switzerland
Bishkek  Kyrgyzstan
Bishkek  Kyrgyzstan
Bissau  Guinea-Bissau
Bissau  Guinea-Bissau


In [458]:
df

,Income ($),city,country,sub-region,region,year,decade,century
0,20900,Port Vila,Vanuatu,Melanesia,Oceania,1930,1920,20
1,9300,Fort-de-France,Martinique,Caribbean,Americas,1936,1930,20
2,5400,Sanaa,Yemen,Western Asia,Asia,1941,1940,20
3,11600,Lome,Togo,Western Africa,Africa,1946,1940,20
4,9300,Luanda,Angola,Middle Africa,Africa,1941,1940,20
...,...,...,...,...,...,...,...,...
15710,9300,Kuala Lumpur,Malaysia,South-Eastern Asia,Asia,1978,1970,20
15711,9300,The Valley,Anguilla,Caribbean,Americas,1862,1860,19
15712,34400,Lesotho,Lesotho,Southern Africa,Africa,1791,1790,18
15713,12400,Accra,Ghana,Western Africa,Africa,1821,1820,19


In [459]:
nova_contagem_df_2 = df.groupby(['city', 'century'])['Income ($)'].count()
nova_contagem_df_2

city            century
 Afghanistan    18          1
                21          1
 Aland Islands  21          1
 Albania        18          2
                21          3
                           ..
Yaren           20         44
Yerevan         19         19
                20         47
Zagreb          19         19
                20         48
Name: Income ($), Length: 827, dtype: int64

In [460]:
novos_val_a_sub_2 = nova_contagem_df_2[nova_contagem_df_2 < K_ANONIM].index
novos_val_a_sub_2

MultiIndex([(              ' Afghanistan', 18),
            (              ' Afghanistan', 21),
            (            ' Aland Islands', 21),
            (                  ' Albania', 18),
            (                  ' Albania', 21),
            (           ' American Samoa', 18),
            (           ' American Samoa', 21),
            (                  ' Andorra', 18),
            (                  ' Andorra', 21),
            (                   ' Angola', 21),
            ...
            ( ' Virgin Islands (British)', 18),
            ( ' Virgin Islands (British)', 19),
            (      ' Virgin Islands (US)', 21),
            (' Wallis And Futuna Islands', 21),
            (           ' Western Sahara', 18),
            (                    ' Yemen', 18),
            (                    ' Yemen', 21),
            (                   ' Zambia', 18),
            (                 ' Zimbabwe', 18),
            (                 ' Zimbabwe', 21)],
           names=['city

for (city, decade) in novos_val_a_sub_2:
    novo_valor_year = df.loc[(df['city'] == city) & (df['decade'] == decade), 'century'].values[0]
    print(year,novo_valor_year)
    df.loc[(df['city'] == city) & (df['decade'] == decade), 'decade'] = novo_valor_year

In [461]:
df

,Income ($),city,country,sub-region,region,year,decade,century
0,20900,Port Vila,Vanuatu,Melanesia,Oceania,1930,1920,20
1,9300,Fort-de-France,Martinique,Caribbean,Americas,1936,1930,20
2,5400,Sanaa,Yemen,Western Asia,Asia,1941,1940,20
3,11600,Lome,Togo,Western Africa,Africa,1946,1940,20
4,9300,Luanda,Angola,Middle Africa,Africa,1941,1940,20
...,...,...,...,...,...,...,...,...
15710,9300,Kuala Lumpur,Malaysia,South-Eastern Asia,Asia,1978,1970,20
15711,9300,The Valley,Anguilla,Caribbean,Americas,1862,1860,19
15712,34400,Lesotho,Lesotho,Southern Africa,Africa,1791,1790,18
15713,12400,Accra,Ghana,Western Africa,Africa,1821,1820,19


In [462]:
nova_contagem_df_3 = df.groupby(['city', 'century'])['Income ($)'].count()
nova_contagem_df_3

city            century
 Afghanistan    18          1
                21          1
 Aland Islands  21          1
 Albania        18          2
                21          3
                           ..
Yaren           20         44
Yerevan         19         19
                20         47
Zagreb          19         19
                20         48
Name: Income ($), Length: 827, dtype: int64

In [463]:
novos_val_a_sub_3 = nova_contagem_df_3[nova_contagem_df_3 < K_ANONIM].index
novos_val_a_sub_3

MultiIndex([(              ' Afghanistan', 18),
            (              ' Afghanistan', 21),
            (            ' Aland Islands', 21),
            (                  ' Albania', 18),
            (                  ' Albania', 21),
            (           ' American Samoa', 18),
            (           ' American Samoa', 21),
            (                  ' Andorra', 18),
            (                  ' Andorra', 21),
            (                   ' Angola', 21),
            ...
            ( ' Virgin Islands (British)', 18),
            ( ' Virgin Islands (British)', 19),
            (      ' Virgin Islands (US)', 21),
            (' Wallis And Futuna Islands', 21),
            (           ' Western Sahara', 18),
            (                    ' Yemen', 18),
            (                    ' Yemen', 21),
            (                   ' Zambia', 18),
            (                 ' Zimbabwe', 18),
            (                 ' Zimbabwe', 21)],
           names=['city

In [464]:
for (city, century) in novos_val_a_sub_3:
    novo_valor_city = df.loc[(df['city'] == city) & (df['century'] == century), 'sub-region'].values[0]
    print(city,novo_valor_city)
    df.loc[(df['city'] == city) & (df['century'] == century), 'city'] = novo_valor_city

 Afghanistan  Southern Asia
 Afghanistan  Southern Asia
 Aland Islands  Northern Europe
 Albania  Southern Europe
 Albania  Southern Europe
 American Samoa  Polynesia
 American Samoa  Polynesia
 Andorra  Southern Europe
 Andorra  Southern Europe
 Angola  Middle Africa
 Anguilla  Caribbean
 Anguilla  Caribbean
 Antigua And Barbuda  Caribbean
 Argentina  South America
 Argentina  South America
 Armenia  Western Asia
 Armenia  Western Asia
 Aruba  Caribbean
 Aruba  Caribbean
 Australia  Australia and New Zealand
 Australia  Australia and New Zealand
 Austria  Western Europe
 Azerbaijan  Western Asia
 Azerbaijan  Western Asia
 Bahamas The  Caribbean
 Bahamas The  Caribbean
 Bahrain  Western Asia
 Bangladesh  Southern Asia
 Barbados  Caribbean
 Barbados  Caribbean
 Belarus  Eastern Europe
 Belarus  Eastern Europe
 Belgium  Western Europe
 Belgium  Western Europe
 Belize  Central America
 Belize  Central America
 Bermuda  Northern America
 Bhutan  Southern Asia
 Bhutan  Southern Asia
 Bolivi

 United Kingdom  Northern Europe
 United States  Northern America
 Uruguay  South America
 Vanuatu  Melanesia
 Vanuatu  Melanesia
 Vatican City State (Holy See)  Southern Europe
 Venezuela  South America
 Vietnam  South-Eastern Asia
 Vietnam  South-Eastern Asia
 Virgin Islands (British)  Caribbean
 Virgin Islands (British)  Caribbean
 Virgin Islands (US)  Caribbean
 Wallis And Futuna Islands  Polynesia
 Western Sahara  Northern Africa
 Yemen  Western Asia
 Yemen  Western Asia
 Zambia  Eastern Africa
 Zimbabwe  Eastern Africa
 Zimbabwe  Eastern Africa


In [465]:
nova_contagem_df_4 = df.groupby(['city', 'century'])['Income ($)'].count()
nova_contagem_df_4

city                        century
 Australia and New Zealand  18          4
                            21         11
 Caribbean                  18         43
                            19          8
                            21         47
                                       ..
Yaren                       20         44
Yerevan                     19         19
                            20         47
Zagreb                      19         19
                            20         48
Name: Income ($), Length: 531, dtype: int64

In [466]:
novos_val_a_sub_4 = nova_contagem_df_4[nova_contagem_df_4 < K_ANONIM].index
novos_val_a_sub_4

MultiIndex([(' Australia and New Zealand', 18),
            (          ' Central America', 19),
            (             ' Central Asia', 18),
            (             ' Central Asia', 21),
            (           ' Eastern Africa', 19),
            (             ' Eastern Asia', 18),
            (           ' Eastern Europe', 19),
            (                ' Melanesia', 18),
            (                ' Melanesia', 21),
            (               ' Micronesia', 19),
            (            ' Middle Africa', 21),
            (          ' Northern Africa', 18),
            (          ' Northern Africa', 19),
            (          ' Northern Africa', 21),
            (         ' Northern America', 18),
            (         ' Northern America', 21),
            (          ' Southern Africa', 18),
            (          ' Southern Africa', 21),
            (           ' Western Europe', 21)],
           names=['city', 'century'])

In [467]:
for (city, century) in novos_val_a_sub_4:
    novo_valor_city = df.loc[(df['city'] == city) & (df['century'] == century), 'region'].values[0]
    print(city,novo_valor_city)
    df.loc[(df['city'] == city) & (df['century'] == century), 'city'] = novo_valor_city

 Australia and New Zealand  Oceania
 Central America  Americas
 Central Asia  Asia
 Central Asia  Asia
 Eastern Africa  Africa
 Eastern Asia  Asia
 Eastern Europe  Europe
 Melanesia  Oceania
 Melanesia  Oceania
 Micronesia  Oceania
 Middle Africa  Africa
 Northern Africa  Africa
 Northern Africa  Africa
 Northern Africa  Africa
 Northern America  Americas
 Northern America  Americas
 Southern Africa  Africa
 Southern Africa  Africa
 Western Europe  Europe


In [468]:
nova_contagem_df_5 = df.groupby(['city', 'century'])['Income ($)'].count()
nova_contagem_df_5

city       century
 Africa    18         11
           19         11
           21         19
 Americas  18          4
           19          4
                      ..
Yaren      20         44
Yerevan    19         19
           20         47
Zagreb     19         19
           20         48
Name: Income ($), Length: 525, dtype: int64

In [469]:
novos_val_a_sub_5 = nova_contagem_df_5[nova_contagem_df_5 < K_ANONIM].index
novos_val_a_sub_5

MultiIndex([(' Americas', 18),
            (' Americas', 19),
            (' Americas', 21),
            (    ' Asia', 21),
            (  ' Europe', 19),
            (  ' Europe', 21),
            ( ' Oceania', 19),
            ( ' Oceania', 21)],
           names=['city', 'century'])

for (city, year) in novos_val_a_sub_5:
    if city == ' Africa':
        novo_valor_year = '19-20'
        print(year,novo_valor_year)
        df.loc[(df['city'] == city) & (df['year'] == year), 'year'] = novo_valor_year

In [483]:
for _, row in df[['city', 'century', 'region']].iterrows():
    if row['city'] == ' Americas' and (row['century'] == 18 or row['century'] == 19 or row['century'] == 20 or row['century'] == 21):
        print(row['city'], ' Asia')
        df.loc[(df['city'] == row['city']) & (df['century'] == row['century']), 'century'] = '18-21'
#         df.loc[(df['city'] == row['city']) & (df['century'] == row['century']), 'city'] = ' Asia'

 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia
 Americas  Asia


In [484]:
nova_contagem_df_6 = df.groupby(['city', 'century'])['Income ($)'].count()
nova_contagem_df_6

city       century
 Africa    18         11
           19         11
           21         19
 Americas  18-21      15
 Asia      18         12
                      ..
Yaren      20         44
Yerevan    19         19
           20         47
Zagreb     19         19
           20         48
Name: Income ($), Length: 517, dtype: int64

In [485]:
novos_val_a_sub_6 = nova_contagem_df_6[nova_contagem_df_6 < K_ANONIM].index
novos_val_a_sub_6

MultiIndex([], names=['city', 'century'])

In [486]:
path_name = str(K_ANONIM) + '_city_century.csv'
df[['city','century','Income ($)']].to_csv(path_name)